In [1]:
!pip install parse-ingredients

In [5]:
# Libraries needed to get the html from a site and to parse the html
from bs4 import BeautifulSoup as bs
import requests
import numpy as np
import csv
from parse_ingredients import parse_ingredient

In [6]:
# Saving the url to a variable, getting the url and parsing the html
mainURL = "https://www.sbs.com.au/food/cuisine/maltese"
req = requests.get(mainURL)
soup = bs(req.text, "html.parser")

In [7]:
# Base link that hrefs will be added onto
baseLink = 'https://www.sbs.com.au'

In [8]:
# The tag that stores the href to the recipes
hrefs = soup.findAll(class_='link-underlay')

In [9]:
# Simply printing out the contents to see what the hrefs of the tags are

def printHrefs(hrefsIn):
    for i in hrefsIn:
        print(i.get('href'), "\n")

# printHrefs(hrefs)

In [10]:
# This for loop will generate the links to all the recipes that are shown on the mainURL

def funcRecipeLinks(hrefsIn, baseLinkIn, recipeLinks):
    for i in hrefsIn:
        fullLink = baseLinkIn + i.get('href')
        if i.get('href')[6] == 'r':
            recipeLinks.append(fullLink)
    return recipeLinks

recipeLinks = []
recipeLinks = funcRecipeLinks(hrefs, baseLink, recipeLinks)

In [11]:
mainURL = "https://www.sbs.com.au/food/cuisine/maltese?sort_by=created&page=1"
req = requests.get(mainURL)
soup = bs(req.text, "html.parser")

baseLink = 'https://www.sbs.com.au'
hrefs = soup.findAll(class_='link-underlay')

# printHrefs(hrefs)

recipeLinks = funcRecipeLinks(hrefs, baseLink, recipeLinks)


In [12]:
# Printing the links to all the recipes shown before clicking the show more button
print(recipeLinks)
print(len(recipeLinks))

['https://www.sbs.com.au/food/recipes/maltese-baked-ricotta-pie', 'https://www.sbs.com.au/food/recipes/maltese-style-bruschetta-hobz-biz-zejt', 'https://www.sbs.com.au/food/recipes/maltese-rabbit-stew-stuffat-tal-fenek', 'https://www.sbs.com.au/food/recipes/bread-oil-hobz-biz-zejt', 'https://www.sbs.com.au/food/recipes/maltese-easter-biscuits-figolli', 'https://www.sbs.com.au/food/recipes/timpana', 'https://www.sbs.com.au/food/recipes/broad-bean-dip-bigilla', 'https://www.sbs.com.au/food/recipes/pears-baked-white-wine-almonds-and-armaretto-langas-mimli-bil-lewz', 'https://www.sbs.com.au/food/recipes/apple-and-fig-cake', 'https://www.sbs.com.au/food/recipes/pork-and-pea-pastizzi', 'https://www.sbs.com.au/food/recipes/rabbit-tomato-and-wine-sauce-rice', 'https://www.sbs.com.au/food/recipes/peach-and-almond-crumble', 'https://www.sbs.com.au/food/recipes/christmas-chocolate-salami-romblu-tal-milied-bic-cikkulata', 'https://www.sbs.com.au/food/recipes/pumpkin-and-rice-pie', 'https://www.sbs

In [15]:
#Test
#result = parse_ingredient("12 ounces lean ground beef, preferably 85 percent lean")
#print(f"Found results: \n {result}")

# Included Attributes: name, quantity, unit, comment, original_string

Found results: 
 Ingredient(name='lean ground beef', quantity=12.0, unit='oz', comment='preferably 85 percent lean', original_string='12 ounces lean ground beef, preferably 85 percent lean')


In [24]:
data = []
ingredientList = []

for recipe in recipeLinks:
    req = requests.get(recipe) # Accesses the next recipe
    soup = bs(req.text, "html.parser")
    title = soup.findAll('h1') # finds the name of the recipe
    
    ingredientsDiv = soup.findAll('div', class_='field-name-field-ingredients') # finds all the divs containing the ingredients
    
    # For each div go trough and extract the ingredients
    for ul in ingredientsDiv:
        ingredients = ul.findAll("li")
        
        for i in ingredients:
            for j in title:
                data.append(str(j.get_text(strip=False))) # Dish Name
                
            parseResult = parse_ingredient(str(i.get_text(strip=False)))
            
            data.append(parseResult.original_string) #Original String
            data.append(parseResult.name) #Ingredient
            data.append(parseResult.quantity) #Quantity
            data.append(parseResult.unit) #Unit
            data.append(parseResult.comment) #Comment
            
            ingredientList.append(data)   
            data = []

In [25]:
import csv

header = ['Dish Name', 'Original String', 'Ingredient', 'Quantity', 'Unit', 'Comment']


with open('CSV/sbsRecipes.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write multiple rows
    writer.writerows(ingredientList)